In [1]:
%config IPCompleter.greedy=True

# Import the dependencies.
import json
import re
import time

import numpy as np
import pandas as pd
import requests

# https://beautiful-soup-4.readthedocs.io/en/latest/
from bs4 import BeautifulSoup as bs

In [2]:
season_id = 2020
url = f"https://www.basketball-reference.com/leagues/NBA_{season_id}_per_game.html"
response = requests.get(url=url)
soup = bs(response.content, "html.parser")

In [3]:
soup.findAll("tr", limit=2)

[<tr>
 <th aria-label="Rank" class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th>
 <th aria-label="Player" class="poptip sort_default_asc center" data-stat="player" scope="col">Player</th>
 <th aria-label="Position" class="poptip sort_default_asc center" data-stat="pos" data-tip="Position" scope="col">Pos</th>
 <th aria-label="Player's age on February 1 of the season" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th>
 <th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_id" data-tip="Team" scope="col">Tm</th>
 <th aria-label="Games" class="poptip center" data-stat="g" data-tip="Games" scope="col">G</th>
 <th aria-label="Games Started" class="poptip center" data-stat="gs" data-tip="Games Started" scope="col">GS</th>
 <th aria-label="Minutes Played Per Game" class="poptip hide_non_quals center" data-stat="mp_per_g"

In [4]:
column_headers = [th.getText() for th in soup.findAll("tr", limit=2)[0].findAll("th")]
print(column_headers)
column_headers = column_headers[1:]

['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


In [5]:
data_rows = soup.findAll("tr")[1:]

player_data = [
    [td.getText() for td in data_rows[i].findAll("td")] for i in range(len(data_rows))
]
print(len(player_data))
#print(player_data)

player_data_filtered = list(filter(None, player_data))
print(len(player_data_filtered))
#print(player_data_filtered)

player_ids = [
    [td.get("data-append-csv") for td in data_rows[i].findAll("td", attrs={"data-append-csv": True})] for i in range(len(data_rows))
]
print(len(player_ids))
#print(player_ids)

flat_list = [item for sublist in player_ids for item in sublist]
print(len(flat_list))
#print(flat_list)

677
651
677
651


In [6]:
general_2019_20_df = pd.DataFrame(player_data_filtered, columns=column_headers)
general_2019_20_df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,.592,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,.557,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,.493,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,1.0,.500,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,.368,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7


In [7]:
# We want to drop the NaN rows
# How to select rows with NaN in particular column?
# df.loc[df['Col2'].isnull()] 
na_reference_df = general_2019_20_df.loc[general_2019_20_df["Player"].isnull()]
na_reference_df

# Dropped rows where the column Player is equal to NaN
general_2019_20_df = general_2019_20_df.loc[~general_2019_20_df["Player"].isin([np.nan])]
general_2019_20_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,.592,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,.557,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,.493,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,1.0,.500,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,.368,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,.437,...,.860,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6
647,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,.524,...,.682,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1
648,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,.250,...,,1.5,0.5,2.0,0.0,0.0,0.0,0.0,0.0,1.0
649,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,.569,...,.737,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4


In [8]:
# Now add the season date to match the format from NBA Stats at the beginning of the data frame
general_2019_20_df.insert(0, "season_id", "2019-20")
general_2019_20_df.head()

,season_id,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019-20,Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,2019-20,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,2019-20,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,2019-20,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,1.0,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,2019-20,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7


In [9]:
# Now add the player_id
general_2019_20_df.insert(1, "bbref_player_id", flat_list)
general_2019_20_df.head()

,season_id,bbref_player_id,Player,Pos,Age,Tm,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019-20,adamsst01,Steven Adams,C,26,OKC,63,63,26.7,4.5,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,2019-20,adebaba01,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,2019-20,aldrila01,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,2019-20,alexaky01,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,2019-20,alexani01,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7


In [10]:
# rename the matching columns
rename_these_columns = {
    "Tm": "team_abbreviation",
    "Age": "age",
    "G": "gp",
    "MP": "min",
    "FG": "fgm",
    "FGA": "fga",
    "FG%": "fg_pct",
    "3P": "fg3m",
    "3PA": "fg3a",
    "3P%": "fg3_pct",
    "FT": "ftm",
    "FTA": "fta",
    "FT%": "ft_pct",
    "ORB": "oreb",
    "DRB": "dreb",
    "TRB": "reb",
    "AST": "ast",
    "TOV": "tov",
    "STL": "stl",
    "BLK": "blk",
    "PF": "pf",
    "PTS": "pts",
}

general_2019_20_df = general_2019_20_df.rename(columns=rename_these_columns, errors="raise")
general_2019_20_df = general_2019_20_df.rename(str.lower, axis='columns')
general_2019_20_df.head()

,season_id,bbref_player_id,player,pos,age,team_abbreviation,gp,gs,min,fgm,...,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts
0,2019-20,adamsst01,Steven Adams,C,26,OKC,63,63,26.7,4.5,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,2019-20,adebaba01,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,2019-20,aldrila01,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,2019-20,alexaky01,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,2019-20,alexani01,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7


In [11]:
# Save to CSV
general_2019_20_df.to_csv(
    "../../data/interim/bbref-general-traditional-2019-20.csv", index=False
)

In [12]:
url_adv = f"https://www.basketball-reference.com/leagues/NBA_{season_id}_advanced.html"
response = requests.get(url=url_adv)
soup = bs(response.content, "html.parser")

In [13]:
soup.findAll("tr", limit=2)

[<tr>
 <th aria-label="Rank" class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th>
 <th aria-label="Player" class="poptip sort_default_asc center" data-stat="player" scope="col">Player</th>
 <th aria-label="Position" class="poptip sort_default_asc center" data-stat="pos" data-tip="Position" scope="col">Pos</th>
 <th aria-label="Player's age on February 1 of the season" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th>
 <th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_id" data-tip="Team" scope="col">Tm</th>
 <th aria-label="Games" class="poptip center" data-stat="g" data-tip="Games" scope="col">G</th>
 <th aria-label="Minutes Played" class="poptip center" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
 <th aria-label="Player Efficiency Rating" class="poptip hide_non_quals center" data-stat="per" d

In [14]:
column_headers = [th.getText() for th in soup.findAll("tr", limit=2)[0].findAll("th")]
column_headers = column_headers[1:]

In [15]:
data_rows = soup.findAll("tr")[1:]

player_data = [
    [td.getText() for td in data_rows[i].findAll("td")] for i in range(len(data_rows))
]
print(len(player_data))
#print(player_data)

player_data_filtered = list(filter(None, player_data))
print(len(player_data_filtered))
#print(player_data_filtered)

player_ids = [
    [td.get("data-append-csv") for td in data_rows[i].findAll("td", attrs={"data-append-csv": True})] for i in range(len(data_rows))
]
print(len(player_ids))
#print(player_ids)

flat_list = [item for sublist in player_ids for item in sublist]
print(len(flat_list))
#print(flat_list)

677
651
677
651


In [16]:
advanced_2019_20_df = pd.DataFrame(player_data_filtered, columns=column_headers)
advanced_2019_20_df.head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Steven Adams,C,26,OKC,63,1680,20.5,.604,.006,.421,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,.018,.484,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,.198,.241,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,Kyle Alexander,C,23,MIA,2,13,4.7,.500,.000,.000,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,.500,.139,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4


In [17]:
# We want to drop the NaN rows
# How to select rows with NaN in particular column?
# df.loc[df['Col2'].isnull()] 
advanced_na_reference_df = advanced_2019_20_df.loc[advanced_2019_20_df["Player"].isnull()]
advanced_na_reference_df

# Dropped rows where the column Player is equal to NaN
advanced_2019_20_df = advanced_2019_20_df.loc[~advanced_2019_20_df["Player"].isin([np.nan])]
advanced_2019_20_df

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Steven Adams,C,26,OKC,63,1680,20.5,.604,.006,.421,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,.018,.484,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,.198,.241,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,Kyle Alexander,C,23,MIA,2,13,4.7,.500,.000,.000,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,.500,.139,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Trae Young,PG,21,ATL,60,2120,23.9,.595,.455,.448,...,,5.3,0.6,5.9,.133,,6.2,-2.3,3.9,3.1
647,Cody Zeller,C,27,CHO,58,1341,18.8,.576,.157,.374,...,,2.3,1.3,3.6,.129,,0.2,-0.8,-0.6,0.5
648,Tyler Zeller,C,30,SAS,2,4,22.4,.250,.000,.000,...,,0.0,0.0,0.0,-0.075,,-0.3,-22.1,-22.4,0.0
649,Ante Žižić,C,23,CLE,22,221,16.4,.597,.000,.264,...,,0.3,0.2,0.5,.106,,-1.7,-1.5,-3.2,-0.1


In [18]:
# Now add the season date to match the format from NBA Stats at the beginning of the data frame
advanced_2019_20_df.insert(0, "season_id", "2019-20")
advanced_2019_20_df.head()

,season_id,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,2019-20,Steven Adams,C,26,OKC,63,1680,20.5,.604,.006,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,2019-20,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,.018,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,2019-20,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,.198,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,2019-20,Kyle Alexander,C,23,MIA,2,13,4.7,.500,.000,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,2019-20,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,.500,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4


In [19]:
# Now add the player_id
advanced_2019_20_df.insert(1, "bbref_player_id", flat_list)
advanced_2019_20_df.head()

,season_id,bbref_player_id,Player,Pos,Age,Tm,G,MP,PER,TS%,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,2019-20,adamsst01,Steven Adams,C,26,OKC,63,1680,20.5,.604,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,2019-20,adebaba01,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,2019-20,aldrila01,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,2019-20,alexaky01,Kyle Alexander,C,23,MIA,2,13,4.7,.500,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,2019-20,alexani01,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4


In [20]:
# Figure out Columns
print(advanced_2019_20_df.columns.values.tolist())

['season_id', 'bbref_player_id', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', '\xa0', 'OWS', 'DWS', 'WS', 'WS/48', '\xa0', 'OBPM', 'DBPM', 'BPM', 'VORP']


In [21]:
# rename the matching columns
rename_these_columns = {
    "Tm": "team_abbreviation",
    "G": "gp",
    "MP": "min",
}

advanced_2019_20_df = advanced_2019_20_df.rename(columns=rename_these_columns, errors="raise")
advanced_2019_20_df = advanced_2019_20_df.rename(str.lower, axis='columns')
advanced_2019_20_df.head()

,season_id,bbref_player_id,player,pos,age,team_abbreviation,gp,min,per,ts%,...,,ows,dws,ws,ws/48,,obpm,dbpm,bpm,vorp
0,2019-20,adamsst01,Steven Adams,C,26,OKC,63,1680,20.5,.604,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,2019-20,adebaba01,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,2019-20,aldrila01,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,2019-20,alexaky01,Kyle Alexander,C,23,MIA,2,13,4.7,.500,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,2019-20,alexani01,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4


In [25]:
# Save to CSV
advanced_2019_20_df.to_csv(
    "../../data/interim/bbref-general-advanced-2019-20.csv", index=False
)

In [26]:
general_2019_20_df = general_2019_20_df[general_2019_20_df["player"].notnull()]

general_2019_20_df = general_2019_20_df[:].fillna(0)

general_2019_20_df = general_2019_20_df.drop_duplicates(["player"], keep="first")

In [27]:
advanced_2019_20_df = advanced_2019_20_df[advanced_2019_20_df["player"].notnull()]

advanced_2019_20_df = advanced_2019_20_df[:].fillna(0)

advanced_2019_20_df = advanced_2019_20_df.drop_duplicates(["player"], keep="first")

In [28]:
complete_2019_20_stats_df = pd.merge(
    general_2019_20_df, advanced_2019_20_df, on="player"
)

In [29]:
complete_2019_20_stats_df

,season_id_x,bbref_player_id_x,player,pos_x,age_x,team_abbreviation_x,gp_x,gs,min_x,fgm,...,,ows,dws,ws,ws/48,,obpm,dbpm,bpm,vorp
0,2019-20,adamsst01,Steven Adams,C,26,OKC,63,63,26.7,4.5,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,2019-20,adebaba01,Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,2019-20,aldrila01,LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,2019-20,alexaky01,Kyle Alexander,C,23,MIA,2,0,6.5,0.5,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,2019-20,alexani01,Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,2019-20,youngtr01,Trae Young,PG,21,ATL,60,60,35.3,9.1,...,,5.3,0.6,5.9,.133,,6.2,-2.3,3.9,3.1
525,2019-20,zelleco01,Cody Zeller,C,27,CHO,58,39,23.1,4.3,...,,2.3,1.3,3.6,.129,,0.2,-0.8,-0.6,0.5
526,2019-20,zellety01,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,...,,0.0,0.0,0.0,-0.075,,-0.3,-22.1,-22.4,0.0
527,2019-20,zizican01,Ante Žižić,C,23,CLE,22,0,10.0,1.9,...,,0.3,0.2,0.5,.106,,-1.7,-1.5,-3.2,-0.1


In [30]:
# Save to CSV
complete_2019_20_stats_df.to_csv(
    "../../data/interim/bbref-general-complete-2019-20.csv", index=False
)